### I- Modèle de Vicsek

code non vectorisé

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from matplotlib.patches import Circle

def simulation2D(bruit_phi,N,L):
    if not os.path.exists('frames'):
        os.makedirs('frames')
    v0=2
    T=2500 #nombre d'itérations
    dt=1
    rayon_interaction=5
    nouvelles_vitesses_x=np.zeros(N)
    nouvelles_vitesses_y=np.zeros(N)

    ordre=0
    
    tabx,taby=np.zeros(N),np.zeros(N)
    tabvx,tabvy=np.zeros(N),np.zeros(N)
    norme=np.zeros(N)
    eta_phi=np.zeros(N)
    historique_phi=[]
    for i in range(N):
        tabx[i],taby[i]=random.randint(0,L-1),random.randint(0,L-1)
        tabvx[i],tabvy[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme[i]=np.sqrt(tabvx[i]**2+tabvy[i]**2)
        tabvx[i],tabvy[i]=tabvx[i]/norme[i],tabvy[i]/norme[i]
    
#########################################################################
    for t in range(T):
        for i in range(len(tabx)):
            x,y=tabx[i],taby[i] #position de particule i
            vx,vy=tabvx[i],tabvy[i] #vitesse de particule i
            indices_particules_proches=[]
            for j in range(len(tabx)):
                xj,yj=tabx[j],taby[j]
                dx=np.abs(x-xj)
                dy=np.abs(y-yj)
                if dx>L/2:
                    dx=L-dx
                if dy>L/2:
                    dy=L-dy 
                distance=np.sqrt(dx**2+dy**2)
                if distance<rayon_interaction:
                    indices_particules_proches.append(j) #indices de toutes les particules j proches de i
            vxj,vyj=tabvx[indices_particules_proches],tabvy[indices_particules_proches] #tableau de toutes les vitesses des particules j 
            sum_vxj,sum_vyj=np.sum(vxj),np.sum(vyj) #somme des composantes selon x et y
            norme_sumj=np.sqrt(sum_vxj**2+sum_vyj**2) #norme du VECTEUR=somme de tous les vecteurs
            nouvelles_vitesses_x[i]=sum_vxj/norme_sumj*v0 #direction moyenne des voisins, normalisée puis * v0
            nouvelles_vitesses_y[i]=sum_vyj/norme_sumj*v0
            
            
        #actualisation synchrone de toutes les vitesses
        tabvx=nouvelles_vitesses_x*1
        tabvy=nouvelles_vitesses_y*1 #*1 pour pas que les modifications de nouvelles_vitesses_xy se fassent aussi dans tabvxy

        for i in range(len(tabvx)):
        #ajoute bruit pour particule i
            V=np.sqrt(tabvy[i]**2+tabvx[i]**2) #norme de V_i
            phi=np.arctan2(tabvy[i],tabvx[i]) #angle entre vx_i et vy_i
            nouveau_phi=(phi+random.uniform(-bruit_phi/2,bruit_phi/2))%(np.pi*2) #ajoute bruit à l'angle
        #nouvelles composantes de vitesse
            tabvx[i]=v0*np.cos(nouveau_phi)
            tabvy[i]=v0*np.sin(nouveau_phi)
        
        #actualisation des positions CONDITIONS PERIODIQUES
        tabx=(tabx+tabvx*dt)%L
        taby=(taby+tabvy*dt)%L

    #calcul ordre
        vitesse_tout_x=np.sum(tabvx)
        vitesse_tout_y=np.sum(tabvy)
        ordre=np.sqrt(vitesse_tout_x**2+vitesse_tout_y**2)/(len(tabvx)*v0)
        historique_phi.append(ordre)
    return historique_phi
    
    

T_TRANSITOIRE=1000 #Temps ignoré pour la stabilisation
N_POINTS_BRUIT=30
tab_bruit=np.linspace(0,np.pi*2,N_POINTS_BRUIT)
tab_ordre=[]

tab_L=[1,5,10,15,20]
tab_N=[5,125,500,1125,2000]  #N/L^2=cste=5 donc N=5*L^2

for j in range(len(tab_L)):
    L=tab_L[j]
    N=tab_N[j]
    tab_ordre_courant=[]    
    for i in range(N_POINTS_BRUIT):
        bruit=tab_bruit[i]        
        res=simulation2D(bruit,N,L) #renvoie historique_phi
        #moyenne sur la partie stationnaire
        phi_moyen=np.mean(res[T_TRANSITOIRE:]) #moyenne sur toutes les valeurs de temps supérieur à T_TRANSITOIRE
        tab_ordre_courant.append(phi_moyen) #ajoute de la valeur de l'ordre (ordre=phi) pour la i_ème valeur de N_POINTS_BRUITS
    
    #tracé de la courbe de tous les ordres pour tous les bruits pour le N
    plt.plot(tab_bruit,tab_ordre_courant,'o-',label = "N="+str(N))

plt.xlabel("Bruit eta")
plt.ylabel("Ordre phi")
plt.legend()
plt.grid(True)
plt.savefig("resultat_final_lent.png")
plt.show()

code partiellement vectorisé (N varie à rho constant)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

def simulation2D(bruit_phi,N,L):
    v0=0.5
    T=500 #nombre d'itérations
    dt=1
    rayon_interaction=1
    nouvelles_vitesses_x=np.zeros(N)
    nouvelles_vitesses_y=np.zeros(N)

    ordre=0
    
    tabx,taby=np.zeros(N),np.zeros(N)
    tabvx,tabvy=np.zeros(N),np.zeros(N)
    norme=np.zeros(N)
    eta_phi=np.zeros(N)
    historique_phi=[]
    for i in range(N):
        tabx[i],taby[i]=random.randint(0,L-1),random.randint(0,L-1)
        tabvx[i],tabvy[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme[i]=np.sqrt(tabvx[i]**2+tabvy[i]**2)
        tabvx[i],tabvy[i]=tabvx[i]/norme[i],tabvy[i]/norme[i]
    
#########################################################################
    for t in range(T):
        for i in range(N):
            dx=np.abs(tabx-tabx[i]) #tableau de diff selon x entre toutes les particules et la particule 1
            dy=np.abs(taby-taby[i])

            dx=np.where(dx>L/2,L-dx,dx) #np.where(condition,valeur_si_vrai,valeur_si_faux)
            dy=np.where(dy>L/2,L-dy,dy) #si dx>L/2 alors c plus proche de passer de l'autre côté: dx --> L-dx

            distances=np.sqrt(dx**2+dy**2) #tableau de dimension 1 de N éléments = distances les + courtes entre la particule i et toutes les autres

            masque_voisins=distances<rayon_interaction #tableau de bouléens: True si dist entre poissons<rayon interaction
            
            sum_vx=np.sum(tabvx[masque_voisins]) #somme des vitesses des poissons dans le cercle
            sum_vy=np.sum(tabvy[masque_voisins])
            norme_sum=np.sqrt(sum_vx**2+sum_vy**2)
            
            if norme_sum!=0: #au cas où la sommes des vitesses donne le vecteur nul
              nouvelles_vitesses_x[i]=sum_vx/norme_sum*v0
              nouvelles_vitesses_y[i]=sum_vy/norme_sum*v0 
            else:
              nouvelles_vitesses_x[i]=tabvx[i]
              nouvelles_vitesses_y[i]=tabvy[i]
          
        tabvx=nouvelles_vitesses_x*1 #en dehors de la boucle et diff tableau pour actualisation synchrone de toutes les vitesses
        tabvy=nouvelles_vitesses_y*1 #*1 pour pas que les modifications de nouvelles_vitesses_xy se fassent aussi dans tabvxy
        #donc là on a le tableau de toutes les vitesses après interaction avec voisins
        
        angles=np.arctan2(nouvelles_vitesses_y,nouvelles_vitesses_x) #renvoie l'angle, le theta au temps t quoi
        bruit=np.random.uniform(-bruit_phi/2, bruit_phi/2, N) #ajout du bruit. crée un tableau de N valeurs de random valeur
        nouveaux_angles=angles+bruit
        
        #mise à jour vitesses
        tabvx=v0*np.cos(nouveaux_angles)
        tabvy=v0*np.sin(nouveaux_angles)
        
        #mise à jour positions + Conditions périodiques
        tabx=(tabx+tabvx*dt)%L
        taby=(taby+tabvy*dt)%L

        #calcul ordre
        vitesse_tout_x=np.sum(tabvx)
        vitesse_tout_y=np.sum(tabvy)
        ordre=np.sqrt(vitesse_tout_x**2+vitesse_tout_y**2)/(N*v0)
        historique_phi.append(ordre)
    return historique_phi

T_TRANSITOIRE=200 #temps ignoré pour la stabilisation
N_POINTS_BRUIT=20 #nombre de valeurs de bruit testées
tab_bruit=np.linspace(0,np.pi*2,N_POINTS_BRUIT) #valeurs de bruit testées
tab_ordre=[]

tab_L = [10, 20, 30, 40]
tab_N = [500, 2000, 4500, 8000]  # N = 5*L^2


for j in range(len(tab_L)):
    L=tab_L[j]
    N=tab_N[j]
    # IMPORTANT : On vide la liste pour CHAQUE courbe???????
    tab_ordre_courant=[]    
    for i in range(N_POINTS_BRUIT):
        bruit=tab_bruit[i]        
        res=simulation2D(bruit,N,L) #renvoie historique_phi
        #moyenne sur la partie stationnaire
        phi_moyen=np.mean(res[T_TRANSITOIRE:]) #moyenne sur toutes les valeurs de temps supérieur à T_TRANSITOIRE
        tab_ordre_courant.append(phi_moyen) #ajoute de la valeur de l'ordre (ordre=phi) pour la i_ème valeur de N_POINTS_BRUITS
    
    #tracé de la courbe de tous les ordres pour tous les bruits pour le N
    plt.plot(tab_bruit,tab_ordre_courant,'o-',label = "N="+str(N))

plt.xlabel("Bruit eta")
plt.ylabel("Ordre phi")
plt.legend()
plt.grid(True)
plt.savefig("resultat_final_lent.png")
plt.show()

code partiellement vectorisé (rho qui varie)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

def simulation2D(bruit_phi,N,L):
    v0=0.5
    T=500 #nombre d'itérations
    dt=1
    rayon_interaction=1
    nouvelles_vitesses_x=np.zeros(N)
    nouvelles_vitesses_y=np.zeros(N)

    ordre=0
    
    tabx,taby=np.zeros(N),np.zeros(N)
    tabvx,tabvy=np.zeros(N),np.zeros(N)
    norme=np.zeros(N)
    eta_phi=np.zeros(N)
    historique_phi=[]
    for i in range(N):
        tabx[i],taby[i]=random.randint(0,L-1),random.randint(0,L-1)
        tabvx[i],tabvy[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme[i]=np.sqrt(tabvx[i]**2+tabvy[i]**2)
        tabvx[i],tabvy[i]=tabvx[i]/norme[i],tabvy[i]/norme[i]
    
#########################################################################
    for t in range(T):
        for i in range(N):
            dx=np.abs(tabx-tabx[i]) #tableau de diff selon x entre toutes les particules et la particule 1
            dy=np.abs(taby-taby[i])

            dx=np.where(dx>L/2,L-dx,dx) #np.where(condition,valeur_si_vrai,valeur_si_faux)
            dy=np.where(dy>L/2,L-dy,dy) #si dx>L/2 alors c plus proche de passer de l'autre côté: dx --> L-dx

            distances=np.sqrt(dx**2+dy**2) #tableau de dimension 1 de N éléments = distances les + courtes entre la particule i et toutes les autres

            masque_voisins=distances<rayon_interaction #tableau de bouléens: True si dist entre poissons<rayon interaction
            
            sum_vx=np.sum(tabvx[masque_voisins]) #somme des vitesses des poissons dans le cercle
            sum_vy=np.sum(tabvy[masque_voisins])
            norme_sum=np.sqrt(sum_vx**2+sum_vy**2)
            
            if norme_sum!=0: #au cas où la sommes des vitesses donne le vecteur nul
              nouvelles_vitesses_x[i]=sum_vx/norme_sum*v0
              nouvelles_vitesses_y[i]=sum_vy/norme_sum*v0 
            else:
              nouvelles_vitesses_x[i]=tabvx[i]
              nouvelles_vitesses_y[i]=tabvy[i]
          
        tabvx=nouvelles_vitesses_x*1 #en dehors de la boucle et diff tableau pour actualisation synchrone de toutes les vitesses
        tabvy=nouvelles_vitesses_y*1 #*1 pour pas que les modifications de nouvelles_vitesses_xy se fassent aussi dans tabvxy
        #donc là on a le tableau de toutes les vitesses après interaction avec voisins
        
        angles=np.arctan2(nouvelles_vitesses_y,nouvelles_vitesses_x) #renvoie l'angle, le theta au temps t quoi
        bruit=np.random.uniform(-bruit_phi/2, bruit_phi/2, N) #ajout du bruit. crée un tableau de N valeurs de random valeur
        nouveaux_angles=angles+bruit
        
        #mise à jour vitesses
        tabvx=v0*np.cos(nouveaux_angles)
        tabvy=v0*np.sin(nouveaux_angles)
        
        #mise à jour positions + Conditions périodiques
        tabx=(tabx+tabvx*dt)%L
        taby=(taby+tabvy*dt)%L

        #calcul ordre
        vitesse_tout_x=np.sum(tabvx)
        vitesse_tout_y=np.sum(tabvy)
        ordre=np.sqrt(vitesse_tout_x**2+vitesse_tout_y**2)/(N*v0)
        historique_phi.append(ordre)
    return historique_phi

T_TRANSITOIRE=200 #temps ignoré pour la stabilisation
N_POINTS_BRUIT=20 #nombre de valeurs de bruit testées
tab_bruit=np.linspace(0,np.pi*2,N_POINTS_BRUIT) #valeurs de bruit testées
tab_ordre=[]

tab_L=[10,10,10]
tab_N=[50,200,500]


for j in range(len(tab_L)):
    L=tab_L[j]
    N=tab_N[j]
    # IMPORTANT : On vide la liste pour CHAQUE courbe???????
    tab_ordre_courant=[]    
    for i in range(N_POINTS_BRUIT):
        bruit=tab_bruit[i]        
        res=simulation2D(bruit,N,L) #renvoie historique_phi
        #moyenne sur la partie stationnaire
        phi_moyen=np.mean(res[T_TRANSITOIRE:]) #moyenne sur toutes les valeurs de temps supérieur à T_TRANSITOIRE
        tab_ordre_courant.append(phi_moyen) #ajoute de la valeur de l'ordre (ordre=phi) pour la i_ème valeur de N_POINTS_BRUITS
    
    #tracé de la courbe de tous les ordres pour tous les bruits pour le N
    plt.plot(tab_bruit,tab_ordre_courant,'o-',label = "N="+str(N))

plt.xlabel("Bruit eta")
plt.ylabel("Ordre phi")
plt.legend()
plt.grid(True)
plt.savefig("resultat_final_lent.png")
plt.show()

code partiellement vectorisé (fluctuations pour $N=50$)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

def simulation2D(bruit_phi,N,L):
    v0=0.5
    T=500 #nombre d'itérations
    dt=1
    rayon_interaction=1
    nouvelles_vitesses_x=np.zeros(N)
    nouvelles_vitesses_y=np.zeros(N)

    ordre=0
    
    tabx,taby=np.zeros(N),np.zeros(N)
    tabvx,tabvy=np.zeros(N),np.zeros(N)
    norme=np.zeros(N)
    eta_phi=np.zeros(N)
    historique_phi=[]
    for i in range(N):
        tabx[i],taby[i]=random.randint(0,L-1),random.randint(0,L-1)
        tabvx[i],tabvy[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme[i]=np.sqrt(tabvx[i]**2+tabvy[i]**2)
        tabvx[i],tabvy[i]=tabvx[i]/norme[i],tabvy[i]/norme[i]
    
#########################################################################
    for t in range(T):
        for i in range(N):
            dx=np.abs(tabx-tabx[i]) #tableau de diff selon x entre toutes les particules et la particule 1
            dy=np.abs(taby-taby[i])

            dx=np.where(dx>L/2,L-dx,dx) #np.where(condition,valeur_si_vrai,valeur_si_faux)
            dy=np.where(dy>L/2,L-dy,dy) #si dx>L/2 alors c plus proche de passer de l'autre côté: dx --> L-dx

            distances=np.sqrt(dx**2+dy**2) #tableau de dimension 1 de N éléments = distances les + courtes entre la particule i et toutes les autres

            masque_voisins=distances<rayon_interaction #tableau de bouléens: True si dist entre poissons<rayon interaction
            
            sum_vx=np.sum(tabvx[masque_voisins]) #somme des vitesses des poissons dans le cercle
            sum_vy=np.sum(tabvy[masque_voisins])
            norme_sum=np.sqrt(sum_vx**2+sum_vy**2)
            
            if norme_sum!=0: #au cas où la sommes des vitesses donne le vecteur nul
              nouvelles_vitesses_x[i]=sum_vx/norme_sum*v0
              nouvelles_vitesses_y[i]=sum_vy/norme_sum*v0 
            else:
              nouvelles_vitesses_x[i]=tabvx[i]
              nouvelles_vitesses_y[i]=tabvy[i]
          
        tabvx=nouvelles_vitesses_x*1 #en dehors de la boucle et diff tableau pour actualisation synchrone de toutes les vitesses
        tabvy=nouvelles_vitesses_y*1 #*1 pour pas que les modifications de nouvelles_vitesses_xy se fassent aussi dans tabvxy
        #donc là on a le tableau de toutes les vitesses après interaction avec voisins
        
        angles=np.arctan2(nouvelles_vitesses_y,nouvelles_vitesses_x) #renvoie l'angle, le theta au temps t quoi
        bruit=np.random.uniform(-bruit_phi/2, bruit_phi/2, N) #ajout du bruit. crée un tableau de N valeurs de random valeur
        nouveaux_angles=angles+bruit
        
        #mise à jour vitesses
        tabvx=v0*np.cos(nouveaux_angles)
        tabvy=v0*np.sin(nouveaux_angles)
        
        #mise à jour positions + Conditions périodiques
        tabx=(tabx+tabvx*dt)%L
        taby=(taby+tabvy*dt)%L

        #calcul ordre
        vitesse_tout_x=np.sum(tabvx)
        vitesse_tout_y=np.sum(tabvy)
        ordre=np.sqrt(vitesse_tout_x**2+vitesse_tout_y**2)/(N*v0)
        historique_phi.append(ordre)
    return historique_phi

T_TRANSITOIRE=200 #temps ignoré pour la stabilisation
N_POINTS_BRUIT=30 #nombre de valeurs de bruit testées
tab_bruit=np.linspace(0,np.pi*2,N_POINTS_BRUIT) #valeurs de bruit testées
tab_ordre=[]

tab_L=[10,10,10,10,10]
tab_N=[50,50,50,50,50]


for j in range(len(tab_L)):
    L=tab_L[j]
    N=tab_N[j]
    # IMPORTANT : On vide la liste pour CHAQUE courbe???????
    tab_ordre_courant=[]    
    for i in range(N_POINTS_BRUIT):
        bruit=tab_bruit[i]        
        res=simulation2D(bruit,N,L) #renvoie historique_phi
        #moyenne sur la partie stationnaire
        phi_moyen=np.mean(res[T_TRANSITOIRE:]) #moyenne sur toutes les valeurs de temps supérieur à T_TRANSITOIRE
        tab_ordre_courant.append(phi_moyen) #ajoute de la valeur de l'ordre (ordre=phi) pour la i_ème valeur de N_POINTS_BRUITS
    
    #tracé de la courbe de tous les ordres pour tous les bruits pour le N
    plt.plot(tab_bruit,tab_ordre_courant,'o-',label = "N="+str(N))

plt.xlabel("Bruit eta")
plt.ylabel("Ordre phi")
plt.legend()
plt.grid(True)
plt.savefig("resultat_final_lent.png")
plt.show()

### II- Extension du modèle de Vicsek: Sélection naturelle.

Code biaisé. Test requin qui chasse argmax râté.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from matplotlib.patches import Circle
from scipy.spatial import cKDTree

phi_max=2*np.pi
def simulation2D(bruit_phi=phi_max):
    if not os.path.exists('frames'):
        os.makedirs('frames')
    v0=2
    L=40 #taille côté espace
    r_reproduction=2*L #en gros 1 poisson peut avoir un bébé avec un poisson de l'autre bout de la map pour éviter les biais
    T=1000 #nombre d'itérations
    dt=1
    N=300 #nombre particules
    comidos=0 #nombre de mangés
    nombre_bebes=1
    comidos=0 #nombre de mangés
    rayon_interaction=5
    histogramme_bruit=[]
    
    tabx,taby=np.zeros(N),np.zeros(N)
    tabvx,tabvy=np.zeros(N),np.zeros(N)
    norme=np.zeros(N)
    eta_phi=np.zeros(N)
    for i in range(N):
        tabx[i],taby[i]=random.randint(0,L-1),random.randint(0,L-1)
        tabvx[i],tabvy[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme[i]=np.sqrt(tabvx[i]**2+tabvy[i]**2)
        tabvx[i],tabvy[i]=tabvx[i]/norme[i],tabvy[i]/norme[i]
        eta_phi[i]=random.uniform(-bruit_phi/2,bruit_phi/2)
    
########################################################################
#🦈mouvement requin
########################################################################
    
    #nombre requins
    Nr=7
    #rayon du champ de vision
    Rvue=10 #rayon de perception du requin arbitrairement 1 dixieme de l'espace. 2*L: tous les poissons sont visibles
    #vitesse requins
    vr=2*v0
    #coordonnées et vitesses des requins
    tabxr,tabyr=np.ones(Nr),np.ones(Nr)
    tabvxr,tabvyr=np.ones(Nr),np.ones(Nr)
    
    for i in range(Nr):
        tabxr[i],tabyr[i]=random.uniform(0,L-1),random.uniform(0,L-1)
        tabvxr[i],tabvyr[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme=np.sqrt(tabvxr[i]**2+tabvyr[i]**2)
        tabvxr[i],tabvyr[i]=tabvxr[i]/norme*vr,tabvyr[i]/norme*vr #pour faire que tous les requins aillent à la vitesse vr
        #tab_eta_phi_r[i]=random.uniform(-bruit_phi/2,bruit_phi/2)
        
    #si a=0 alors requin tranquil, si a=1 requin énervé qui chasse
    a=np.zeros(Nr)
    comidos=np.zeros(Nr)
    indice_solo=np.ones(Nr,dtype=int)*-1 #-1 pour indiquer que pas de poisson ciblé et qd a[r]==0 (le requin choisit une cible), dtype=int sinon c des float dedans
    cibles_oranges=np.zeros((Nr,2))

    for t in range(T):
        poissons_manges_cette_etape=[]
        for r in range(Nr):
            xr,yr=tabxr[r],tabyr[r]
            vxr,vyr=tabvxr[r],tabvyr[r]
            distances_requin=np.sqrt((tabx-xr)**2+(taby-yr)**2) #distances entre TOUS les P et le R
            indices_visibles=np.where(distances_requin<Rvue)[0] #indices des poissons dans champs de vision du R

            if len(indices_visibles)==0:
                #il bouge tranquille
                tabxr[r]+=dt*tabvxr[r]
                tabyr[r]+=dt*tabvyr[r]
                #si il voit rien alors il arrête de chasser si il chassait avant
                a[r]=0
                indice_solo[r]=-1 #le requin a plus de cible
                cibles_oranges[r,:]=0 #on dessine pas en orange de poisson pour le requin r
                continue #toute la suite est ignorée et on passe au requin suivant
            
            #positions des P visibles par R
            visible_r_x=tabx[indices_visibles]
            visible_r_y=taby[indices_visibles] #CE SONT TOUS LES POISSONS DANS LE GRAND CERCLE ROUGE DE IMAGE1.png 

            #différences entre tous les poissons, ancienne double boucle
            x_vis=visible_r_x
            y_vis=visible_r_y
            dx=x_vis[:,None]-x_vis[None,:]
            dy=y_vis[:,None]-y_vis[None,:]
            dist2=dx**2+dy**2 #dist2: matrice de distance. contient les distances entre CHAQUE paire de P visibles
            
            #Compter les voisins dans le rayon 
            #On regarde quel poisson est le plus seul dans le champ de vision du requin
            rayon_densite_poissons=Rvue/4 #divisé par 4 pour calculer la densité "locale" de poissons. Rvue: petit cercle bleu de l'image1.png
            nombre_poissons_autour=np.sum(dist2<rayon_densite_poissons**2,axis=1) #mesure de solitude locale, cb de poisson y'a dans cercle bleu

            if a[r]==0:
                #Détection du poisson le plus solo
                #indice_solo_LOCAL est l'index dans le sous-ensemble indices_visibles!!!!!!!!!!
                #si on a 100 poissons et R voit les P aux positions 12, 13, 50 et 99 dans le tableau tabx alors indices_visibles=[12, 13, 50, 99]
                indice_solo_LOCAL=np.argmin(nombre_poissons_autour) #INDICE_SOLO PARMIS LA LISTE DES POISSONS PROCHES DONC FAUT TROUVER L'INDICE DE CE POISSON DANS LA LISTE TOTALE DE TOUS LES POISSONS!!!!!
                #en haut c'est le poisson jaune ciblé dans image1.png.
                indice_cible_final=indices_visibles[indice_solo_LOCAL] #indice du poisson, dans la liste totale, qui est le + seul
                indice_solo[r]=indice_cible_final
                dx=tabx[indice_solo[r]]-xr
                dy=taby[indice_solo[r]]-yr
                a[r]=1
                #Vecteur reliant le requin et le poisson solo
                dist_min=np.sqrt(dx**2+dy**2)
                tabvxr[r],tabvyr[r]=dx/dist_min*vr,dy/dist_min*vr #composantes vecteurs unitaires du requin au poisson
                tabxr[r],tabyr[r]=xr+vxr*dt,yr+vyr*dt
                if np.sqrt(dx**2+dy**2)<5: #si poissons à 5m alors il meurt
                    #tabx,taby=np.delete(tabx,indice_solo),np.delete(taby,indice_solo) #On a plus N mais N-1
                    #tabvx,tabvy=np.delete(tabvx,indice_solo),np.delete(tabvy,indice_solo) #On enlève la vitesse de la particule mangée
                    #On peut pas enlever parceque sinon les indices vont changer et donc les requins chassent d'autres indices, etc.
                    poissons_manges_cette_etape.append(indice_solo[r]) #append et pas remplace valeur car on sait pas cb y'en a
                    comidos[r]+=1
                    a[r]=0
                    indice_solo[r]=-1

            if a[r]==1: #si il a detecté un poisson auparavant
                #Vecteur reliant le requin et le poisson solo
                dx=tabx[indice_solo[r]]-xr
                dy=taby[indice_solo[r]]-yr
                dist_min=np.sqrt(dx**2+dy**2)
                tabvxr[r],tabvyr[r]=dx/dist_min*vr,dy/dist_min*vr #composantes vecteurs unitaires du requin au poisson *vr
                tabxr[r],tabyr[r]=xr+vxr*dt,yr+vyr*dt
                if np.sqrt(dx**2+dy**2)<2: #si poissons à 2m alors il meurt
                    #tabx,taby=np.delete(tabx,indice_solo),np.delete(taby,indice_solo) #On a plus N mais N-1
                    #tabvx,tabvy=np.delete(tabvx,indice_solo),np.delete(tabvy,indice_solo) #On enlève la vitesse de la particule mangée
                    poissons_manges_cette_etape.append(indice_solo[r]) 
                    comidos[r]+=1
                    a[r]=0
                    indice_solo[r]=-1 #Cible est consommée donc plus de cible
                    
                if a[r]==1 and indice_solo[r]<len(tabx):
                    # Met à jour la position de la cible uniquement si elle n'est pas encore mangée
                    cibles_oranges[r,0]=tabx[indice_solo[r]]
                    cibles_oranges[r,1]=taby[indice_solo[r]]
                elif a[r]==0:
                    cibles_oranges[r,:]=0 # Cible mangée, on affiche pas
                
            if comidos[r]==1:
                comidos[r],a[r]=0,0
                
#######################################################
#GESTION DES SUPPRESSIONS EN BLOC
#######################################################
        if poissons_manges_cette_etape:
            # Assurer des indices uniques et triés par ordre décroissant pour une suppression sûre
            indices_a_supprimer = sorted(list(set(poissons_manges_cette_etape)), reverse=True)
            # 1. Correction des indices de cible des requins EN CHASSE
            
            for r in range(Nr):
                if a[r] == 1 and indice_solo[r]!=-1:
                    # Compter combien d'indices inférieurs à la cible seront supprimés
                    decalage=0
                    for indice in indices_a_supprimer:
                         if indice_solo[r]>indice:
                             decalage+=1
                             
                    indice_solo[r]=indice_solo[r]-decalage

            # 2. Suppression des poissons dans les tableaux globaux (du plus grand indice au plus petit)
            # On utilise np.delete sur un tableau d'indices pour le faire en une seule fois
            tabx=np.delete(tabx,indices_a_supprimer)
            taby=np.delete(taby,indices_a_supprimer)
            tabvx=np.delete(tabvx,indices_a_supprimer)
            tabvy=np.delete(tabvy,indices_a_supprimer)
            eta_phi=np.delete(eta_phi,indices_a_supprimer)

        
###### NAISSANCE ENFANTS ################################################
        nombre_nouveau_nés=len(poissons_manges_cette_etape)
        if nombre_nouveau_nés!=0:
            rayon_reproduction=np.zeros((len(tabx),4)) #4 colonnes: particule i, particule j, distance entre 2
            for _ in range(nombre_nouveau_nés):
        # Choisir deux parents aléatoirement qui sont proches
                parents_trouves=0
                while parents_trouves==0:
                    i,j=np.random.choice(len(tabx),size=2,replace=False)
                    distance=np.sqrt((tabx[i]-tabx[j])**2+(taby[i]-taby[j])**2)
                    if distance<=r_reproduction:
                        parents_trouves=1
                choix_parent=np.random.choice([0,1]) #le bébé spawn aléatoirement sur poisson 1 ou 2
                if choix_parent==0:
                    position_x=tabx[i]
                    position_y=taby[i]
                    vx_bebe=tabvx[i]
                    vy_bebe=tabvy[i]
                else:
                    position_x=tabx[j]
                    position_y=taby[j]
                    vx_bebe=tabvx[j]
                    vy_bebe=tabvy[j]

                sigma_mutation=0.1 #exemple
                mutation=np.random.normal(loc=0,scale=sigma_mutation)
                eta_phi_bebe=(eta_phi[i]+eta_phi[j])/2+mutation #nouveau bruit avec mutation
                eta_phi_bebe=np.clip(eta_phi_bebe,-np.pi,np.pi) #np.clip pour rester dans les bornes de -pi,pi
            #Le bruit η est centré en 0. La moyenne de deux parents très bruités
        # (ex: +π et -π) peut donner un enfant avec η=0,
        # ce qui n'est pas cohérent.
            #C'est l'origine du biais, discuté dans le rapport

                
                tabx=np.append(tabx,position_x)
                taby=np.append(taby,position_y)
                eta_phi=np.append(eta_phi,eta_phi_bebe)
                tabvx=np.append(tabvx,vx_bebe)
                tabvy=np.append(tabvy,vy_bebe)
        
#########################################################################

        #positions des particules
        positions=np.column_stack((tabx,taby))  #forme (N,2)
        #construire l'arbre
        tree=cKDTree(positions)
        #rayon d'interaction
        rayon_interaction=5
        # récupérer pour chaque particule les indices des voisins
        indices_voisins=tree.query_ball_tree(tree,r=rayon_interaction)
        #indices_voisins[i] est une liste des indices j voisins de la particule i
        #mise à jour des vitesses
        for i, voisins in enumerate(indices_voisins):
            sum_vx=np.sum(tabvx[voisins])
            sum_vy=np.sum(tabvy[voisins])
            norme=np.sqrt(sum_vx**2+sum_vy**2)
            tabvx[i]=sum_vx/norme*v0
            tabvy[i]=sum_vy/norme*v0
        
    
            #ajoute bruit pour particule i
            V=np.sqrt(tabvy[i]**2+tabvx[i]**2) #norme de V_i
            phi=np.arctan2(tabvy[i],tabvx[i]) #angle entre vx_i et vy_i
            nouveau_phi=(phi+random.uniform(-eta_phi[i]/2,eta_phi[i]/2))%(np.pi*2) #ajoute bruit à l'angle 
            #nouvelles composantes de vitesse
            tabvx[i]=v0*np.cos(nouveau_phi)
            tabvy[i]=v0*np.sin(nouveau_phi)
            
        #actualisation des positions CONDITIONS PERIODIQUES
        tabx=tabx+tabvx*dt
        taby=taby+tabvy*dt

        #BUT DES NON BOUCLES:
        #Étape 1 : Création du Masque. On crée un masque qui dit pour chaque poisson s'il a franchi la limite.
        #Étape 2 : Indexation Avancée. On utilise ce masque pour n'appliquer les corrections de position et de vitesse qu'aux poissons qui ont le statut True (c'est-à-dire, ceux qui sont sortis).
        # 1. Gestion du Bord X>L
        cond1=tabx>L
        tabx[cond1]=L-(tabx[cond1]-L) #Position ajustée
        tabvx[cond1]*=-1 #Vitesse inversée
        # 2. Gestion du Bord X < 0
        cond2=tabx<0
        tabx[cond2]=np.abs(tabx[cond2]) #Position ajustée
        tabvx[cond2]*=-1 #Vitesse inversée
        # 3. Gestion du Bord Y > L
        cond3=taby>L
        taby[cond3]=L-(taby[cond3]-L) #Position ajustée
        tabvy[cond3]*=-1 #Vitesse inversée
        # 4. Gestion du Bord Y < 0
        cond4=taby<0
        taby[cond4]=np.abs(taby[cond4]) #Position ajustée
        tabvy[cond4]*=-1 #Vitesse inversée

        #MEMES CONDITIONS POUR LES REQUINS
         #BUT DES NON BOUCLES:
        #Étape 1 : Création du Masque. On crée un masque qui dit pour chaque poisson s'il a franchi la limite.
        #Étape 2 : Indexation Avancée. On utilise ce masque pour n'appliquer les corrections de position et de vitesse qu'aux poissons qui ont le statut True (c'est-à-dire, ceux qui sont sortis).
        # 1. Gestion du Bord X>L
        cond1r=tabxr>L
        tabxr[cond1r]=L-(tabxr[cond1r]-L) #Position ajustée
        tabvxr[cond1r]*=-1 #Vitesse inversée
        # 2. Gestion du Bord X < 0
        cond2r=tabxr<0
        tabxr[cond2r]=np.abs(tabxr[cond2r]) #Position ajustée
        tabvxr[cond2r]*=-1 #Vitesse inversée
        # 3. Gestion du Bord Y > L
        cond3r=tabyr>L
        tabyr[cond3r]=L-(tabyr[cond3r]-L) #Position ajustée
        tabvyr[cond3r]*=-1 #Vitesse inversée
        # 4. Gestion du Bord Y < 0
        cond4r=tabyr<0
        tabyr[cond4r]=np.abs(tabyr[cond4r]) #Position ajustée
        tabvyr[cond4r]*=-1 #Vitesse inversée

        fig,axs=plt.subplots(ncols=2,nrows=1,figsize=(10,5))
        axs[0].set_xlim(0,L)
        axs[0].set_ylim(0,L)
        #scatter avec la colormap centrée sur zéro
        sc=axs[0].scatter(tabx,taby,s=10,c=eta_phi,cmap='viridis',vmin=-np.pi,vmax=np.pi,label="Poissons")
        fig.colorbar(sc,ax=axs[0],orientation='vertical',label="Valeur du bruit η (-π à π)")
        for r in range(Nr): #afficher le poisson ciblé
            if a[r]==1:
                axs[0].scatter(cibles_oranges[r,0],cibles_oranges[r,1],s=10,c='darkorange')

        #plot requins avec cercles de rayon Rvue pour le champs de vision
        # --- Dessin des REQUINS, de leur ZONE DE VISION et de leur CIBLE ---
        for r in range(Nr):
            xr, yr = tabxr[r], tabyr[r]
            # CRÉATION DU CERCLE POUR LA ZONE DE VISION
            # Le centre du cercle est (xr, yr), son rayon est Rvue.
            circle = Circle((xr, yr), Rvue, facecolor='red', alpha=0.1) #alpha c la transparence
            # AJOUT DU CERCLE AU GRAPHIQUE
            # plt.gca() récupère l'objet graphique actuel (vos axes).
            # .add_patch(circle) ajoute notre cercle à ce graphique.
            axs[0].add_patch(circle)
            # Dessin du REQUIN lui-même (comme un petit triangle rouge)
            # s=30 pour la taille, c="red" pour la couleur, marker='^' pour la forme.
            axs[0].scatter(xr, yr, s=30, c="red", marker='^',label="Requins") #juste le point du requin, pas la zone
        #histogramme bruit
        moyenne_bruit=np.mean(eta_phi)
        ecart_bruit=np.std(eta_phi)
        axs[1].hist(eta_phi,bins=30, color='skyblue', edgecolor='black')
        axs[1].set_title(f"Distribution du Bruit η\nMoyenne={moyenne_bruit:.2f}, Écart-type={ecart_bruit:.2f}")
        axs[1].set_xlabel("Valeur du Bruit (η)")
        axs[1].set_ylabel("Nombre de Poissons")
        #simulation
        axs[0].set_title("Position des poissons et des requins")
        axs[0].set_xlabel("Position x")
        axs[0].set_ylabel("Position y")
        fig.tight_layout()
        fig.savefig(f"frames/frame_{t:03d}.png")
        plt.close(fig)

simulation2D(2*np.pi) #buit max phi: 2pi

Code non biaisé. Requins qui chassent des poissons (différents chacun) et poissons qui ne fuient PAS ni rien les requins. Ils bougent juste selon VM.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from matplotlib.patches import Circle
from scipy.spatial import cKDTree

indice_solo=None
amplitude_bruit_max=np.pi #maintenant le bruit max c'est pi et on fait "pile ou face" pour gauche ou droite. pour enlever pb de signe et que ça soit toujours centré en 0
#maintenant le bruit max c'est pi et on fait "pile ou face" pour gauche ou droite

def simulation2D(amplitude_bruit=amplitude_bruit_max):
    if not os.path.exists('frames'):
        os.makedirs('frames')
    v0=1
    L=40 #taille côté espace
    r_reproduction=2*L #distance maximale pour reproduction
    T=1000 #nombre d'itérations
    dt=1
    N=300 #nombre particules
    comidos=0 #nombre de mangés
    nombre_bebes=1
    comidos=0 #nombre de mangés
    rayon_interaction=5
    histogramme_bruit=[]
    
    tabx,taby=np.zeros(N),np.zeros(N)
    tabvx,tabvy=np.zeros(N),np.zeros(N)
    norme=np.zeros(N)
    eta_phi=np.zeros(N)
    for i in range(N):
        tabx[i],taby[i]=random.randint(0,L-1),random.randint(0,L-1)
        tabvx[i],tabvy[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme[i]=np.sqrt(tabvx[i]**2+tabvy[i]**2)
        tabvx[i],tabvy[i]=tabvx[i]/norme[i],tabvy[i]/norme[i]
        eta_phi[i] = random.uniform(0, amplitude_bruit) #on tire entre 0 et le max (pi). que du positif

    ########################################################################
#🦈mouvement requin
########################################################################
    
    #nombre requins
    Nr=7
    #rayon du champ de vision
    Rvue=10 #rayon de perception du requin arbitrairement 1 dixieme de l'espace. 2*L: tous les poissons sont visibles
    #vitesse requins
    vr=2*v0
    #coordonnées et vitesses des requins
    tabxr,tabyr=np.ones(Nr),np.ones(Nr)
    tabvxr,tabvyr=np.ones(Nr),np.ones(Nr)
    
    for i in range(Nr):
        tabxr[i],tabyr[i]=random.uniform(0,L-1),random.uniform(0,L-1)
        tabvxr[i],tabvyr[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme=np.sqrt(tabvxr[i]**2+tabvyr[i]**2)
        tabvxr[i],tabvyr[i]=tabvxr[i]/norme*vr,tabvyr[i]/norme*vr #pour faire que tous les requins aillent à la vitesse vr
        #tab_eta_phi_r[i]=random.uniform(-bruit_phi/2,bruit_phi/2)
        
    #si a=0 alors requin tranquil, si a=1 requin énervé qui chasse
    a=np.zeros(Nr)
    comidos=np.zeros(Nr)
    indice_solo=np.ones(Nr,dtype=int)*-1 #-1 pour indiquer que pas de poisson ciblé et qd a[r]==0 (le requin choisit une cible), dtype=int sinon c des float dedans
    cibles_oranges=np.zeros((Nr,2))

    for t in range(T):
        poissons_manges_cette_etape=[]
        liste_noire=[]
        liste_recherche_x=tabx*1
        liste_recherche_y=taby*1

        # Si un requin chasse déjà (a=1), sa cible ne doit pas être vue par les autres
        for r in range(Nr):
            if a[r]==1 and indice_solo[r]!=-1:
                # On téléporte la cible existante très loin dans la liste de recherche
                indice=indice_solo[r]
                # Vérification de sécurité pour ne pas planter si l'indice n'existe plus
                if indice<len(liste_recherche_x):
                    liste_recherche_x[indice]=100000
                    liste_recherche_y[indice]=100000


        
        for r in range(Nr):
            xr,yr=tabxr[r],tabyr[r]
            vxr,vyr=tabvxr[r],tabvyr[r]
            distances_requin=np.sqrt((liste_recherche_x-xr)**2+(liste_recherche_y-yr)**2) #distances entre TOUS les P et le R
            indices_visibles=np.where(distances_requin<Rvue)[0] #indices des poissons dans champs de vision du R

            if len(indices_visibles)==0:
                #il bouge tranquille
                tabxr[r]+=dt*tabvxr[r]
                tabyr[r]+=dt*tabvyr[r]
                #si il voit rien alors il arrête de chasser si il chassait avant
                a[r]=0
                indice_solo[r]=-1 #le requin a plus de cible
                cibles_oranges[r,:]=0 #on dessine pas en orange de poisson pour le requin r
                continue #toute la suite est ignorée et on passe au requin suivant
            
            #positions des poissons visibles par le requin
            visible_r_x=tabx[indices_visibles]
            visible_r_y=taby[indices_visibles] 

            #différences entre tous les poissons, ancienne double boucle
            x_vis=visible_r_x
            y_vis=visible_r_y
            dx=x_vis[:,None]-x_vis[None,:]
            dy=y_vis[:,None]-y_vis[None,:]
            dist2=dx**2+dy**2 #dist2: matrice de distance. contient les distances entre CHAQUE paire de P visibles
            
            #Compter les voisins dans le rayon 
            #On regarde quel poisson est le plus seul dans le champ de vision du requin
            rayon_densite_poissons=Rvue/4 #divisé par 4 pour calculer la densité "locale" de poissons. Rvue: petit cercle bleu de l'image1.png
            nombre_poissons_autour=np.sum(dist2<rayon_densite_poissons**2,axis=1) #mesure de solitude locale, cb de poisson y'a dans cercle bleu

            if a[r]==0:
                #Détection du poisson le plus solo
                #indice_solo_LOCAL est l'index dans le sous-ensemble indices_visibles!!!!!!!!!!
                #si on a 100 poissons et R voit les P aux positions 12, 13, 50 et 99 dans le tableau tabx alors indices_visibles=[12, 13, 50, 99]
                indice_solo_LOCAL=np.argmin(nombre_poissons_autour) #INDICE_SOLO PARMIS LA LISTE DES POISSONS PROCHES DONC FAUT TROUVER L'INDICE DE CE POISSON DANS LA LISTE TOTALE DE TOUS LES POISSONS!!!!!
                #en haut c'est le poisson jaune ciblé dans image1.png.
                indice_cible_final=indices_visibles[indice_solo_LOCAL] #indice du poisson, dans la liste totale, qui est le + seul
                indice_solo[r]=indice_cible_final

                liste_recherche_x[indice_cible_final]=100000 #on le met à 10000 pour que le requin suivant ne le voie pas
                liste_recherche_y[indice_cible_final]=100000
                
                dx=tabx[indice_solo[r]]-xr
                dy=taby[indice_solo[r]]-yr
                a[r]=1
                #Vecteur reliant le requin et le poisson solo
                dist_min=np.sqrt(dx**2+dy**2)
                tabvxr[r],tabvyr[r]=dx/dist_min*vr,dy/dist_min*vr #composantes vecteurs unitaires du requin au poisson
                tabxr[r],tabyr[r]=xr+vxr*dt,yr+vyr*dt
                if np.sqrt(dx**2+dy**2)<5: #si poissons à 5m alors il meurt
                    #tabx,taby=np.delete(tabx,indice_solo),np.delete(taby,indice_solo) #On a plus N mais N-1
                    #tabvx,tabvy=np.delete(tabvx,indice_solo),np.delete(tabvy,indice_solo) #On enlève la vitesse de la particule mangée
                    #On peut pas enlever parceque sinon les indices vont changer et donc les requins chassent d'autres indices, etc.
                    poissons_manges_cette_etape.append(indice_solo[r]) #append et pas remplace valeur car on sait pas cb y'en a
                    comidos[r]+=1
                    a[r]=0
                    indice_solo[r]=-1

            if a[r]==1: #si il a detecté un poisson auparavant
                #Vecteur reliant le requin et le poisson solo
                dx=tabx[indice_solo[r]]-xr
                dy=taby[indice_solo[r]]-yr
                dist_min=np.sqrt(dx**2+dy**2)
                tabvxr[r],tabvyr[r]=dx/dist_min*vr,dy/dist_min*vr #composantes vecteurs unitaires du requin au poisson *vr
                tabxr[r],tabyr[r]=xr+vxr*dt,yr+vyr*dt
                if np.sqrt(dx**2+dy**2)<2: #si poissons à 2m alors il meurt
                    #tabx,taby=np.delete(tabx,indice_solo),np.delete(taby,indice_solo) #On a plus N mais N-1
                    #tabvx,tabvy=np.delete(tabvx,indice_solo),np.delete(tabvy,indice_solo) #On enlève la vitesse de la particule mangée
                    poissons_manges_cette_etape.append(indice_solo[r]) 
                    comidos[r]+=1
                    a[r]=0
                    indice_solo[r]=-1 #Cible est consommée donc plus de cible
                    
                if a[r]==1 and indice_solo[r]<len(tabx):
                    # Met à jour la position de la cible uniquement si elle n'est pas encore mangée
                    cibles_oranges[r,0]=tabx[indice_solo[r]]
                    cibles_oranges[r,1]=taby[indice_solo[r]]
                elif a[r]==0:
                    cibles_oranges[r,:]=0 # Cible mangée, on affiche pas
                
            if comidos[r]==1:
                comidos[r],a[r]=0,0

            liste_noire.append(indice_solo)
                
#######################################################
#GESTION DES SUPPRESSIONS EN BLOC
#######################################################
        if poissons_manges_cette_etape:
            # Assurer des indices uniques et triés par ordre décroissant pour une suppression sûre
            indices_a_supprimer = sorted(list(set(poissons_manges_cette_etape)), reverse=True)
            # 1. Correction des indices de cible des requins EN CHASSE
            
            for r in range(Nr):
                if a[r] == 1 and indice_solo[r]!=-1:
                    # Compter combien d'indices inférieurs à la cible seront supprimés
                    decalage=0
                    for indice in indices_a_supprimer:
                         if indice_solo[r]>indice:
                             decalage+=1
                             
                    indice_solo[r]=indice_solo[r]-decalage

            # 2. Suppression des poissons dans les tableaux globaux (du plus grand indice au plus petit)
            # On utilise np.delete sur un tableau d'indices pour le faire en une seule fois
            tabx=np.delete(tabx,indices_a_supprimer)
            taby=np.delete(taby,indices_a_supprimer)
            tabvx=np.delete(tabvx,indices_a_supprimer)
            tabvy=np.delete(tabvy,indices_a_supprimer)
            eta_phi=np.delete(eta_phi,indices_a_supprimer)

        
###### NAISSANCE ENFANTS ################################################
        nombre_nouveau_nés=len(poissons_manges_cette_etape)
        if nombre_nouveau_nés!=0:
            rayon_reproduction=np.zeros((len(tabx),4)) #4 colonnes: particule i, particule j, distance entre 2
            for _ in range(nombre_nouveau_nés):
        # Choisir deux parents aléatoirement qui sont proches
                parents_trouves=0
                while parents_trouves==0:
                    i,j=np.random.choice(len(tabx),size=2,replace=False)
                    distance=np.sqrt((tabx[i]-tabx[j])**2+(taby[i]-taby[j])**2)
                    if distance<=r_reproduction:
                        parents_trouves=1
                choix_parent=np.random.choice([0,1]) #le bébé spawn aléatoirement sur poisson 1 ou 2
                if choix_parent==0:
                    position_x=tabx[i]
                    position_y=taby[i]
                    vx_bebe=tabvx[i]
                    vy_bebe=tabvy[i]
                else:
                    position_x=tabx[j]
                    position_y=taby[j]
                    vx_bebe=tabvx[j]
                    vy_bebe=tabvy[j]

                sigma_mutation=0.1 #exemple
                mutation=np.random.normal(loc=0,scale=sigma_mutation)
                eta_phi_bebe=(eta_phi[i]+eta_phi[j])/2+mutation #nouveau bruit avec mutation
                eta_phi_bebe=np.clip(eta_phi_bebe,0,np.pi) #np.clip pour rester dans les bornes de -pi,pi
                
                tabx=np.append(tabx,position_x)
                taby=np.append(taby,position_y)
                eta_phi=np.append(eta_phi,eta_phi_bebe)
                tabvx=np.append(tabvx,vx_bebe)
                tabvy=np.append(tabvy,vy_bebe)
        
#########################################################################
        nouvelles_vitesses_x=np.zeros(len(tabvx))
        nouvelles_vitesses_y=np.zeros(len(tabvy))
        #positions des particules
        positions=np.column_stack((tabx,taby))  #forme (N,2)
        #construire l'arbre
        tree=cKDTree(positions)
        #rayon d'interaction
        rayon_interaction=5
        # récupérer pour chaque particule les indices des voisins
        indices_voisins=tree.query_ball_tree(tree,r=rayon_interaction)
        #indices_voisins[i] est une liste des indices j voisins de la particule i
        #mise à jour des vitesses
        for i, voisins in enumerate(indices_voisins):
            sum_vx=np.sum(tabvx[voisins])
            sum_vy=np.sum(tabvy[voisins])
            norme=np.sqrt(sum_vx**2+sum_vy**2)
            nouvelles_vitesses_x[i]=sum_vx/norme*v0
            nouvelles_vitesses_y[i]=sum_vy/norme*v0
        tabvx=nouvelles_vitesses_x+0
        tabvy=nouvelles_vitesses_y+0 
        
        for i in range(len(tabvx)): #j'ai du faire une nouvelle boucle car on a sorti de la boucle les 2 dernieres lignes pour que ce soit synchrone
            #ajoute bruit pour particule i
            V=np.sqrt(tabvy[i]**2+tabvx[i]**2) #norme de V_i
            phi=np.arctan2(tabvy[i],tabvx[i]) #angle entre vx_i et vy_i

            #################
            #On récupère l'amplitude MAX autorisée par le gène du poisson
            amplitude_max=eta_phi[i]
            #on tire une déviation au hasard entre 0 et son max
            angle_deviation=random.uniform(0,amplitude_max)
            #on décide ici si c'est gauche (-1) ou droite (+1)
            signe=random.choice([-1,1])
            #on ajoute ça à l'angle moyen
            angle_final=phi+(angle_deviation*signe)
            #si l'angle dépasse 180deg, cos/sin vont gérer tout seuls.
            tabvx[i]=v0*np.cos(angle_final)
            tabvy[i]=v0*np.sin(angle_final)
            #################
            
        #actualisation des positions
        tabx=tabx+tabvx*dt
        taby=taby+tabvy*dt

        #gestion du Bord X>L
        cond1=tabx>L
        tabx[cond1]=L-(tabx[cond1]-L) #Position ajustée
        tabvx[cond1]*=-1 #Vitesse inversée

        cond2=tabx<0
        tabx[cond2]=np.abs(tabx[cond2]) #Position ajustée
        tabvx[cond2]*=-1 #Vitesse inversée

        cond3=taby>L
        taby[cond3]=L-(taby[cond3]-L) #Position ajustée
        tabvy[cond3]*=-1 #Vitesse inversée

        cond4=taby<0
        taby[cond4]=np.abs(taby[cond4]) #Position ajustée
        tabvy[cond4]*=-1 #Vitesse inversée

        #idem requins
        cond1r=tabxr>L
        tabxr[cond1r]=L-(tabxr[cond1r]-L) #Position ajustée
        tabvxr[cond1r]*=-1 #Vitesse inversée

        cond2r=tabxr<0
        tabxr[cond2r]=np.abs(tabxr[cond2r]) #Position ajustée
        tabvxr[cond2r]*=-1 #Vitesse inversée

        cond3r=tabyr>L
        tabyr[cond3r]=L-(tabyr[cond3r]-L) #Position ajustée
        tabvyr[cond3r]*=-1 #Vitesse inversée

        cond4r=tabyr<0
        tabyr[cond4r]=np.abs(tabyr[cond4r]) #Position ajustée
        tabvyr[cond4r]*=-1 #Vitesse inversée


        
        fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(12, 6))
        
        axs[0].set_xlim(0, L)
        axs[0].set_ylim(0, L)
        
        # 1. TOUS LES POISSONS (Flèches colorées par gène)
        q = axs[0].quiver(
            tabx, taby, tabvx, tabvy, eta_phi,
            cmap='viridis',
            pivot='mid',
            
            # --- AJUSTEMENTS POUR TAILLE RÉDUITE ---
            scale=40,          # Flèches plus courtes (demandé)
            width=0.0048,      # Épaisseur réduite (0.01 -> 0.0048) pour garder la proportion
            linewidth=1,     # Contour affiné (1 -> 0.5) pour rester net
            # ---------------------------------------
            
            # On ne touche pas à ces proportions (relatives au width)
            headwidth=3,       
            headlength=2,      
            headaxislength=4.5,
            
            edgecolor='black',
            zorder=1
    )
        indices_cibles = []
        for r in range(Nr):
            if a[r] == 1 and indice_solo[r] != -1 and indice_solo[r] < len(tabx):
                indices_cibles.append(indice_solo[r])
        
        if indices_cibles:
            # On dessine une deuxième couche de flèches uniquement pour eux
            # zorder=10 assure qu'elles sont au premier plan
            axs[0].quiver(tabx[indices_cibles], taby[indices_cibles], 
                          tabvx[indices_cibles], tabvy[indices_cibles], 
                          color='orange', pivot='mid',scale=40,width=0.0048,linewidth=1,headwidth=3,headlength=2,headaxislength=4.5,edgecolor='black',zorder=1)

        fig.colorbar(q, ax=axs[0], orientation='vertical', label="Valeur du bruit η (0 à π)") # --- Dessin des REQUINS, de leur ZONE DE VISION et de leur CIBLE ---
        for r in range(Nr):
            xr, yr = tabxr[r], tabyr[r]
            # CRÉATION DU CERCLE POUR LA ZONE DE VISION
            # Le centre du cercle est (xr, yr), son rayon est Rvue.
            circle = Circle((xr, yr), Rvue, facecolor='red', alpha=0.05) #alpha c la transparence
            # AJOUT DU CERCLE AU GRAPHIQUE
            # plt.gca() récupère l'objet graphique actuel (vos axes).
            # .add_patch(circle) ajoute notre cercle à ce graphique.
            axs[0].add_patch(circle)
            # Dessin du REQUIN lui-même (comme un petit triangle rouge)
            # s=30 pour la taille, c="red" pour la couleur, marker='^' pour la forme.
            axs[0].scatter(xr, yr, s=30, c="red", marker='^',label="Requins") #juste le point du requin, pas la zone
       # Histogramme amélioré
        moyenne_bruit = np.mean(eta_phi)
        ecart_bruit = np.std(eta_phi)
    
        axs[1].hist(
        eta_phi, bins=30, range=(0, np.pi),
        color='#69b3a2',  # couleur plus agréable
        edgecolor='white', # bordures plus douces
        alpha=0.9          # légère transparence
        )
        axs[1].set_title(
        f"Distribution du bruit η\nMoyenne={moyenne_bruit:.2f}, Écart-type={ecart_bruit:.2f}",
        fontsize=14, fontweight='bold'
        )
        axs[1].set_xlabel("Valeur du bruit η", fontsize=12)
        axs[1].set_ylabel("Nombre de poissons", fontsize=12)
        axs[1].grid(axis='y', linestyle='--', alpha=0.7)

        fig.tight_layout()
        fig.savefig(f"frames/frame_{t:04d}.png")
        plt.close(fig) # Ferme la figure pour libérer la mémoire

simulation2D(np.pi) #buit max phi: pi

### III- Dernières améliorations du modèle: introduction des "Boids"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from matplotlib.patches import Circle
from scipy.spatial import cKDTree

indice_solo=None
amplitude_bruit_max=np.pi #maintenant le bruit max c'est pi et on fait "pile ou face" pour gauche ou droite. pour enlever pb de signe et que ça soit toujours centré en 0
def simulation2D(amplitude_bruit=amplitude_bruit_max):
    if not os.path.exists('frames'):
        os.makedirs('frames')
    v0=1
    L=40 #taille côté espace
    r_reproduction=2*L #en gros 1 poisson peut avoir un bébé avec un poisson de l'autre bout de la map pour éviter les biais
    T=1000 #nombre d'itérations
    dt=1
    N=300 #nombre particules
    comidos=0 #nombre de mangés
    nombre_bebes=1
    comidos=0 #nombre de mangés
    rayon_interaction=5
    histogramme_bruit=[]
    
    tabx,taby=np.zeros(N),np.zeros(N)
    tabvx,tabvy=np.zeros(N),np.zeros(N)
    norme=np.zeros(N)
    eta_phi=np.zeros(N)
    for i in range(N):
        tabx[i],taby[i]=random.randint(0,L-1),random.randint(0,L-1)
        tabvx[i],tabvy[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme[i]=np.sqrt(tabvx[i]**2+tabvy[i]**2)
        tabvx[i],tabvy[i]=tabvx[i]/norme[i],tabvy[i]/norme[i]
        eta_phi[i] = random.uniform(0, amplitude_bruit) #on tire entre 0 et le max (pi). que du positif
########################################################################
#🦈mouvement requin
########################################################################
    
    #nombre requins
    Nr=7
    #rayon du champ de vision
    Rvue=10 #rayon de perception du requin arbitrairement 1 dixieme de l'espace. 2*L: tous les poissons sont visibles
    #vitesse requins
    vr=2*v0
    #coordonnées et vitesses des requins
    tabxr,tabyr=np.ones(Nr),np.ones(Nr)
    tabvxr,tabvyr=np.ones(Nr),np.ones(Nr)
    
    for i in range(Nr):
        tabxr[i],tabyr[i]=random.uniform(0,L-1),random.uniform(0,L-1)
        tabvxr[i],tabvyr[i]=random.uniform(-1,1),random.uniform(-1,1)
        norme=np.sqrt(tabvxr[i]**2+tabvyr[i]**2)
        tabvxr[i],tabvyr[i]=tabvxr[i]/norme*vr,tabvyr[i]/norme*vr #pour faire que tous les requins aillent à la vitesse vr
        #tab_eta_phi_r[i]=random.uniform(-bruit_phi/2,bruit_phi/2)
        
    #si a=0 alors requin tranquil, si a=1 requin énervé qui chasse
    a=np.zeros(Nr)
    comidos=np.zeros(Nr)
    indice_solo=np.ones(Nr,dtype=int)*-1 #-1 pour indiquer que pas de poisson ciblé et qd a[r]==0 (le requin choisit une cible), dtype=int sinon c des float dedans
    cibles_oranges=np.zeros((Nr,2))

    for t in range(T):
        poissons_manges_cette_etape=[]
        liste_noire=[]
        liste_recherche_x=tabx*1
        liste_recherche_y=taby*1

        # Si un requin chasse déjà (a=1), sa cible ne doit pas être vue par les autres
        for r in range(Nr):
            if a[r]==1 and indice_solo[r]!=-1:
                # On téléporte la cible existante très loin dans la liste de recherche
                indice=indice_solo[r]
                # Vérification de sécurité pour ne pas planter si l'indice n'existe plus
                if indice<len(liste_recherche_x):
                    liste_recherche_x[indice]=100000
                    liste_recherche_y[indice]=100000


        
        for r in range(Nr):
            xr,yr=tabxr[r],tabyr[r]
            vxr,vyr=tabvxr[r],tabvyr[r]
            distances_requin=np.sqrt((liste_recherche_x-xr)**2+(liste_recherche_y-yr)**2) #distances entre TOUS les P et le R
            indices_visibles=np.where(distances_requin<Rvue)[0] #indices des poissons dans champs de vision du R

            if len(indices_visibles)==0:
                #il bouge tranquille
                tabxr[r]+=dt*tabvxr[r]
                tabyr[r]+=dt*tabvyr[r]
                #si il voit rien alors il arrête de chasser si il chassait avant
                a[r]=0
                indice_solo[r]=-1 #le requin a plus de cible
                cibles_oranges[r,:]=0 #on dessine pas en orange de poisson pour le requin r
                continue #toute la suite est ignorée et on passe au requin suivant
            
            #positions des P visibles par R
            visible_r_x=tabx[indices_visibles]
            visible_r_y=taby[indices_visibles] #CE SONT TOUS LES POISSONS DANS LE GRAND CERCLE ROUGE DE IMAGE1.png 

            #différences entre tous les poissons, ancienne double boucle
            x_vis=visible_r_x
            y_vis=visible_r_y
            dx=x_vis[:,None]-x_vis[None,:]
            dy=y_vis[:,None]-y_vis[None,:]
            dist2=dx**2+dy**2 #dist2: matrice de distance. contient les distances entre CHAQUE paire de P visibles
            
            #Compter les voisins dans le rayon 
            #On regarde quel poisson est le plus seul dans le champ de vision du requin
            rayon_densite_poissons=Rvue/4 #divisé par 4 pour calculer la densité "locale" de poissons. Rvue: petit cercle bleu de l'image1.png
            nombre_poissons_autour=np.sum(dist2<rayon_densite_poissons**2,axis=1) #mesure de solitude locale, cb de poisson y'a dans cercle bleu

            if a[r]==0:
                #Détection du poisson le plus solo
                #indice_solo_LOCAL est l'index dans le sous-ensemble indices_visibles!!!!!!!!!!
                #si on a 100 poissons et R voit les P aux positions 12, 13, 50 et 99 dans le tableau tabx alors indices_visibles=[12, 13, 50, 99]
                indice_solo_LOCAL=np.argmin(nombre_poissons_autour) #INDICE_SOLO PARMIS LA LISTE DES POISSONS PROCHES DONC FAUT TROUVER L'INDICE DE CE POISSON DANS LA LISTE TOTALE DE TOUS LES POISSONS!!!!!
                #en haut c'est le poisson jaune ciblé dans image1.png.
                indice_cible_final=indices_visibles[indice_solo_LOCAL] #indice du poisson, dans la liste totale, qui est le + seul
                indice_solo[r]=indice_cible_final

                liste_recherche_x[indice_cible_final]=100000 #on le met à 10000 pour que le requin suivant ne le voie pas
                liste_recherche_y[indice_cible_final]=100000
                
                dx=tabx[indice_solo[r]]-xr
                dy=taby[indice_solo[r]]-yr
                a[r]=1
                #Vecteur reliant le requin et le poisson solo
                dist_min=np.sqrt(dx**2+dy**2)
                tabvxr[r],tabvyr[r]=dx/dist_min*vr,dy/dist_min*vr #composantes vecteurs unitaires du requin au poisson
                tabxr[r],tabyr[r]=xr+vxr*dt,yr+vyr*dt
                if np.sqrt(dx**2+dy**2)<5: #si poissons à 5m alors il meurt
                    #tabx,taby=np.delete(tabx,indice_solo),np.delete(taby,indice_solo) #On a plus N mais N-1
                    #tabvx,tabvy=np.delete(tabvx,indice_solo),np.delete(tabvy,indice_solo) #On enlève la vitesse de la particule mangée
                    #On peut pas enlever parceque sinon les indices vont changer et donc les requins chassent d'autres indices, etc.
                    poissons_manges_cette_etape.append(indice_solo[r]) #append et pas remplace valeur car on sait pas cb y'en a
                    comidos[r]+=1
                    a[r]=0
                    indice_solo[r]=-1

            if a[r]==1: #si il a detecté un poisson auparavant
                #Vecteur reliant le requin et le poisson solo
                dx=tabx[indice_solo[r]]-xr
                dy=taby[indice_solo[r]]-yr
                dist_min=np.sqrt(dx**2+dy**2)
                tabvxr[r],tabvyr[r]=dx/dist_min*vr,dy/dist_min*vr #composantes vecteurs unitaires du requin au poisson *vr
                tabxr[r],tabyr[r]=xr+vxr*dt,yr+vyr*dt
                if np.sqrt(dx**2+dy**2)<2: #si poissons à 2m alors il meurt
                    #tabx,taby=np.delete(tabx,indice_solo),np.delete(taby,indice_solo) #On a plus N mais N-1
                    #tabvx,tabvy=np.delete(tabvx,indice_solo),np.delete(tabvy,indice_solo) #On enlève la vitesse de la particule mangée
                    poissons_manges_cette_etape.append(indice_solo[r]) 
                    comidos[r]+=1
                    a[r]=0
                    indice_solo[r]=-1 #Cible est consommée donc plus de cible
                    
                if a[r]==1 and indice_solo[r]<len(tabx):
                    # Met à jour la position de la cible uniquement si elle n'est pas encore mangée
                    cibles_oranges[r,0]=tabx[indice_solo[r]]
                    cibles_oranges[r,1]=taby[indice_solo[r]]
                elif a[r]==0:
                    cibles_oranges[r,:]=0 # Cible mangée, on affiche pas
                
            if comidos[r]==1:
                comidos[r],a[r]=0,0



            liste_noire.append(indice_solo)
                
#######################################################
#GESTION DES SUPPRESSIONS EN BLOC
#######################################################
        if poissons_manges_cette_etape:
            # Assurer des indices uniques et triés par ordre décroissant pour une suppression sûre
            indices_a_supprimer = sorted(list(set(poissons_manges_cette_etape)), reverse=True)
            # 1. Correction des indices de cible des requins EN CHASSE
            
            for r in range(Nr):
                if a[r] == 1 and indice_solo[r]!=-1:
                    # Compter combien d'indices inférieurs à la cible seront supprimés
                    decalage=0
                    for indice in indices_a_supprimer:
                         if indice_solo[r]>indice:
                             decalage+=1
                             
                    indice_solo[r]=indice_solo[r]-decalage

            # 2. Suppression des poissons dans les tableaux globaux (du plus grand indice au plus petit)
            # On utilise np.delete sur un tableau d'indices pour le faire en une seule fois
            tabx=np.delete(tabx,indices_a_supprimer)
            taby=np.delete(taby,indices_a_supprimer)
            tabvx=np.delete(tabvx,indices_a_supprimer)
            tabvy=np.delete(tabvy,indices_a_supprimer)
            eta_phi=np.delete(eta_phi,indices_a_supprimer)

        
###### NAISSANCE ENFANTS ################################################
        nombre_nouveau_nés=len(poissons_manges_cette_etape)
        if nombre_nouveau_nés!=0:
            rayon_reproduction=np.zeros((len(tabx),4)) #4 colonnes: particule i, particule j, distance entre 2
            for _ in range(nombre_nouveau_nés):
        # Choisir deux parents aléatoirement qui sont proches
                parents_trouves=0
                while parents_trouves==0:
                    i,j=np.random.choice(len(tabx),size=2,replace=False)
                    distance=np.sqrt((tabx[i]-tabx[j])**2+(taby[i]-taby[j])**2)
                    if distance<=r_reproduction:
                        parents_trouves=1
                choix_parent=np.random.choice([0,1]) #le bébé spawn aléatoirement sur poisson 1 ou 2
                if choix_parent==0:
                    position_x=tabx[i]
                    position_y=taby[i]
                    vx_bebe=tabvx[i]
                    vy_bebe=tabvy[i]
                else:
                    position_x=tabx[j]
                    position_y=taby[j]
                    vx_bebe=tabvx[j]
                    vy_bebe=tabvy[j]

                sigma_mutation=0.1 #exemple
                mutation=np.random.normal(loc=0,scale=sigma_mutation)
                eta_phi_bebe=(eta_phi[i]+eta_phi[j])/2+mutation #nouveau bruit avec mutation
                eta_phi_bebe=np.clip(eta_phi_bebe,0,np.pi) #np.clip pour rester dans les bornes de -pi,pi
                
                tabx=np.append(tabx,position_x)
                taby=np.append(taby,position_y)
                eta_phi=np.append(eta_phi,eta_phi_bebe)
                tabvx=np.append(tabvx,vx_bebe)
                tabvy=np.append(tabvy,vy_bebe)
        
#########################################################################
        nouvelles_vitesses_x=np.zeros(len(tabvx))
        nouvelles_vitesses_y=np.zeros(len(tabvy))
        #positions des particules
        positions=np.column_stack((tabx,taby))  #forme (N,2)
        #construire l'arbre
        tree=cKDTree(positions)
        #rayon d'interaction
        rayon_interaction=5
        # récupérer pour chaque particule les indices des voisins
        indices_voisins=tree.query_ball_tree(tree,r=rayon_interaction)
        #indices_voisins[i] est une liste des indices j voisins de la particule i
        #mise à jour des vitesses
        for i, voisins in enumerate(indices_voisins):
            if len(voisins) <= 1:
                continue #si pas de voisins alors prochain i

            #1. Alignement (truc de base)
            sum_vx=np.sum(tabvx[voisins])
            sum_vy=np.sum(tabvy[voisins])
            norme=np.sqrt(sum_vx**2+sum_vy**2)
            vx_align=sum_vx/norme*v0
            vy_align=sum_vy/norme*v0

            #2. Cohésion (chaque poisson tende vers le centre de son groupe)
            centre_x=np.mean(tabx[voisins])
            centre_y=np.mean(taby[voisins])
            vx_cohesion=(centre_x-tabx[i])
            vy_cohesion=(centre_y-taby[i]) #vecteur qui part du poisson et termine au centre

            # 3. Séparation (Correction anti-crash + Seuil 0.5)
            vx_separation=0
            vy_separation=0
            force_repulsion=0.5 # La force avec laquelle ils se repoussent

            for j in voisins:
                if i==j:
                    continue # pas prendre en compte lui-même

                dx=tabx[i]-tabx[j]
                dy=taby[i]-taby[j]
                distance=np.sqrt(dx**2+dy**2)

                # Si le poisson est dans la zone critique (< 0.5)
                if distance<0.5: 
                    
                    # PROTECTION CRITIQUE: Si distance est presque 0 (évite la division par 0)
                    if distance<0.0001:
                        #On donne une impulsion au hasard pour les décoincer
                        vx_separation+=random.uniform(-1,1)*force_repulsion
                        vy_separation+=random.uniform(-1,1)*force_repulsion
                    
                    # CAS STANDARD: On repousse proprement
                    else:
                        # (dx/distance) donne la direction, * force_repulsion donne la puissance
                        vx_separation += (dx / distance) * force_repulsion
                        vy_separation += (dy / distance) * force_repulsion


            #Combinaison pondérée
            alpha_align=0.6
            alpha_cohesion=0.1
            alpha_separation=0.9

            vx_total=alpha_align*vx_align+alpha_cohesion*vx_cohesion+alpha_separation*vx_separation
            vy_total=alpha_align*vy_align+alpha_cohesion*vy_cohesion+alpha_separation*vy_separation


            
            norme=np.sqrt(vx_total**2+vy_total**2)
            nouvelles_vitesses_x[i]=vx_total/norme*v0
            nouvelles_vitesses_y[i]=vy_total/norme*v0

        tabvx=nouvelles_vitesses_x+0
        tabvy=nouvelles_vitesses_y+0 
        

        for i in range(len(tabvx)): #j'ai du faire une nouvelle boucle car on a sorti de la boucle les 2 dernieres lignes pour que ce soit synchrone
            #ajoute bruit pour particule i
            V=np.sqrt(tabvy[i]**2+tabvx[i]**2) #norme de V_i
            phi=np.arctan2(tabvy[i],tabvx[i]) #angle entre vx_i et vy_i

            #################
            #On récupère l'amplitude MAX autorisée par le gène du poisson
            amplitude_max=eta_phi[i]
            #on tire une déviation au hasard entre 0 et son max
            angle_deviation=random.uniform(0,amplitude_max)
            #on décide ici si c'est gauche (-1) ou droite (+1)
            signe=random.choice([-1,1])
            #on ajoute ça à l'angle moyen
            angle_final=phi+(angle_deviation*signe)
            #si l'angle dépasse 180deg, cos/sin vont gérer tout seuls.
            tabvx[i]=v0*np.cos(angle_final)
            tabvy[i]=v0*np.sin(angle_final)
            #################
            
        #actualisation des positions
        tabx=tabx+tabvx*dt
        taby=taby+tabvy*dt

        #BUT DES NON BOUCLES:
        #Étape 1 : Création du Masque. On crée un masque qui dit pour chaque poisson s'il a franchi la limite.
        #Étape 2 : Indexation Avancée. On utilise ce masque pour n'appliquer les corrections de position et de vitesse qu'aux poissons qui ont le statut True (c'est-à-dire, ceux qui sont sortis).
        # 1. Gestion du Bord X>L
        cond1=tabx>L
        tabx[cond1]=L-(tabx[cond1]-L) #Position ajustée
        tabvx[cond1]*=-1 #Vitesse inversée
        # 2. Gestion du Bord X < 0
        cond2=tabx<0
        tabx[cond2]=np.abs(tabx[cond2]) #Position ajustée
        tabvx[cond2]*=-1 #Vitesse inversée
        # 3. Gestion du Bord Y > L
        cond3=taby>L
        taby[cond3]=L-(taby[cond3]-L) #Position ajustée
        tabvy[cond3]*=-1 #Vitesse inversée
        # 4. Gestion du Bord Y < 0
        cond4=taby<0
        taby[cond4]=np.abs(taby[cond4]) #Position ajustée
        tabvy[cond4]*=-1 #Vitesse inversée

        #MEMES CONDITIONS POUR LES REQUINS
         #BUT DES NON BOUCLES:
        #Étape 1 : Création du Masque. On crée un masque qui dit pour chaque poisson s'il a franchi la limite.
        #Étape 2 : Indexation Avancée. On utilise ce masque pour n'appliquer les corrections de position et de vitesse qu'aux poissons qui ont le statut True (c'est-à-dire, ceux qui sont sortis).
        # 1. Gestion du Bord X>L
        cond1r=tabxr>L
        tabxr[cond1r]=L-(tabxr[cond1r]-L) #Position ajustée
        tabvxr[cond1r]*=-1 #Vitesse inversée
        # 2. Gestion du Bord X < 0
        cond2r=tabxr<0
        tabxr[cond2r]=np.abs(tabxr[cond2r]) #Position ajustée
        tabvxr[cond2r]*=-1 #Vitesse inversée
        # 3. Gestion du Bord Y > L
        cond3r=tabyr>L
        tabyr[cond3r]=L-(tabyr[cond3r]-L) #Position ajustée
        tabvyr[cond3r]*=-1 #Vitesse inversée
        # 4. Gestion du Bord Y < 0
        cond4r=tabyr<0
        tabyr[cond4r]=np.abs(tabyr[cond4r]) #Position ajustée
        tabvyr[cond4r]*=-1 #Vitesse inversée

        # --- AFFICHAGE ---
        fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(12, 6))
        
        axs[0].set_xlim(0, L)
        axs[0].set_ylim(0, L)
        
        # 1. TOUS LES POISSONS (Flèches colorées par gène)
        q = axs[0].quiver(
            tabx, taby, tabvx, tabvy, eta_phi,
            cmap='viridis',
            pivot='mid',
            
            # --- AJUSTEMENTS POUR TAILLE RÉDUITE ---
            scale=40,          # Flèches plus courtes (demandé)
            width=0.0048,      # Épaisseur réduite (0.01 -> 0.0048) pour garder la proportion
            linewidth=1,     # Contour affiné (1 -> 0.5) pour rester net
            # ---------------------------------------
            
            # On ne touche pas à ces proportions (relatives au width)
            headwidth=3,       
            headlength=2,      
            headaxislength=4.5,
            
            edgecolor='black',
            zorder=1
    )
        indices_cibles = []
        for r in range(Nr):
            if a[r] == 1 and indice_solo[r] != -1 and indice_solo[r] < len(tabx):
                indices_cibles.append(indice_solo[r])
        
        if indices_cibles:
            # On dessine une deuxième couche de flèches uniquement pour eux
            # zorder=10 assure qu'elles sont au premier plan
            axs[0].quiver(tabx[indices_cibles], taby[indices_cibles], 
                          tabvx[indices_cibles], tabvy[indices_cibles], 
                          color='orange', pivot='mid',scale=40,width=0.0048,linewidth=1,headwidth=3,headlength=2,headaxislength=4.5,edgecolor='black',zorder=1)

        fig.colorbar(q, ax=axs[0], orientation='vertical', label="Valeur du bruit η (0 à π)") # --- Dessin des REQUINS, de leur ZONE DE VISION et de leur CIBLE ---
        for r in range(Nr):
            xr, yr = tabxr[r], tabyr[r]
            # CRÉATION DU CERCLE POUR LA ZONE DE VISION
            # Le centre du cercle est (xr, yr), son rayon est Rvue.
            circle = Circle((xr, yr), Rvue, facecolor='red', alpha=0.05) #alpha c la transparence
            # AJOUT DU CERCLE AU GRAPHIQUE
            # plt.gca() récupère l'objet graphique actuel (vos axes).
            # .add_patch(circle) ajoute notre cercle à ce graphique.
            axs[0].add_patch(circle)
            # Dessin du REQUIN lui-même (comme un petit triangle rouge)
            # s=30 pour la taille, c="red" pour la couleur, marker='^' pour la forme.
            axs[0].scatter(xr, yr, s=30, c="red", marker='^',label="Requins") #juste le point du requin, pas la zone
       # Histogramme amélioré
        moyenne_bruit = np.mean(eta_phi)
        ecart_bruit = np.std(eta_phi)
    
        axs[1].hist(
        eta_phi, bins=30, range=(0, np.pi),
        color='#69b3a2',  # couleur plus agréable
        edgecolor='white', # bordures plus douces
        alpha=0.9          # légère transparence
        )
        axs[1].set_title(
        f"Distribution du bruit η\nMoyenne={moyenne_bruit:.2f}, Écart-type={ecart_bruit:.2f}",
        fontsize=14, fontweight='bold'
        )
        axs[1].set_xlabel("Valeur du bruit η", fontsize=12)
        axs[1].set_ylabel("Nombre de poissons", fontsize=12)
        axs[1].grid(axis='y', linestyle='--', alpha=0.7)

        fig.tight_layout()
        fig.savefig(f"frames/frame_{t:04d}.png")
        plt.close(fig) # Ferme la figure pour libérer la mémoire

simulation2D(np.pi) #buit max phi: pi

Code pour générer le gif à partir du dossier "frames".

In [ ]:
from PIL import Image
import os

def create_gif_ultra_safe(frames_dir='frames', output_filename='animation.gif', fps=10):

    files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])

    if not files:
        print("Aucune image trouvée.")
        return

    first_frame = None
    frames = []

    duration = int(1000 / fps)  # durée en ms par frame

    for f in files:
        path = os.path.join(frames_dir, f)
        try:
            img = Image.open(path)
            img = img.convert("RGB")

            if first_frame is None:
                first_frame = img
            else:
                frames.append(img)

        except Exception as e:
            print(f"[SKIP] {f} illisible : {e}")

    if first_frame is None:
        print("Aucune image valide.")
        return

    first_frame.save(
        output_filename,
        save_all=True,
        append_images=frames,
        duration=duration,
        loop=0,
        optimize=False
    )
    print(f"✅ GIF créé : {output_filename} ({1 + len(frames)} frames)")
create_gif_ultra_safe()